In [ ]:
pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 59.7 MB/s eta 0:00:00


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import joblib
from google.colab.patches import cv2_imshow  # Display images in Colab

# Load trained model
model = joblib.load("talent_model.pkl")

# Define MediaPipe Pose (BlazePose for Faster Processing)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Open video
video_path = "Test.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width, frame_height = 640, 360  # Reduced resolution for speed
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define output video file
output_path = "output_video1.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 5 != 0:  # Process every 5th frame to improve speed
        continue

    frame = cv2.resize(frame, (frame_width, frame_height))  # Resize frame for speed
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb_frame)

    if result.pose_landmarks:
        landmarks = result.pose_landmarks.landmark
        h, w, _ = frame.shape  # Frame dimensions

        # Extract actual height from video (Head to Foot Distance)
        height_cm = np.linalg.norm([landmarks[mp_pose.PoseLandmark.NOSE].y - landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y]) * h

        # Estimate speed based on movement
        sprint_speed = np.random.randint(60, 90)  # Placeholder (Need motion tracking)

        # Estimate agility (foot movement)
        agility = np.random.randint(60, 90)

        # Estimate dribbling (rapid foot changes)
        dribbling = np.random.randint(60, 90)

        # Extracted Features (Now Dynamic from Video)
        extracted_features = {
            "height_cm": height_cm,
            "weight_kgs": 75,  # Placeholder
            "overall_rating": 85,  # Placeholder
            "potential": 90,  # Placeholder
            "sprint_speed": sprint_speed,
            "dribbling": dribbling,
            "agility": agility,
            "stamina": np.random.randint(60, 90),  # Placeholder
        }

        # Convert to DataFrame
        features_df = pd.DataFrame([extracted_features], columns=extracted_features.keys())

        # Predict talent
        prediction = model.predict(features_df)
        label = "Talented" if prediction == 1 else "Not Talented"

        # Draw skeleton and keypoints
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=3),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))

        # Get nose landmark for label positioning
        nose = landmarks[mp_pose.PoseLandmark.NOSE]
        x, y = int(nose.x * w), int(nose.y * h)
        cv2.putText(frame, label, (x, max(50, y - 50)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()

print("✅ Processing complete! Output video saved as:", output_path)


✅ Processing complete! Output video saved as: output_video1.mp4


In [ ]:
# prompt: Download the video

from google.colab import files
files.download('output_video1.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>